<a href="https://colab.research.google.com/github/myniggname/abednigo.github.io/blob/main/Cohort_Analysis_Bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#allow access on google drive

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
#importing dataset from bigquery
#create new table of author and cohort_month

%%bigquery --project revoufsda-350705 cohort_table

SELECT
author,
MIN(DATE_TRUNC(Date(time_ts),Month)) as Cohort_Month,
FROM `bigquery-public-data.hacker_news.stories`
WHERE EXTRACT( YEAR FROM DATE(time_ts)) = 2014 AND author IS NOT NULL
GROUP BY 1
ORDER BY 2

In [3]:
cohort_table

,author,Cohort_Month
0,marketest,2014-01-01
1,jeassonlens,2014-01-01
2,aryekellman01,2014-01-01
3,divramis,2014-01-01
4,learnlivegrow,2014-01-01
...,...,...
57492,tolmasky,2014-12-01
57493,yakattack,2014-12-01
57494,friendly_chap,2014-12-01
57495,armish,2014-12-01


In [9]:
#Define active month and Return Period User

%%bigquery --project revoufsda-350705 active_table

#Define cohort table
WITH cohort_table AS(
SELECT
author,
MIN(DATE_TRUNC(Date(time_ts),Month)) as cohort_month,
FROM `bigquery-public-data.hacker_news.stories`
WHERE EXTRACT( YEAR FROM DATE(time_ts)) = 2014 AND author IS NOT NULL
GROUP BY 1
ORDER BY 2
),

#Define active month and Return Period User
active_table AS(
SELECT
stories.author AS author,
DATE_TRUNC(Date(stories.time_ts), Month) as active_month,
cohort_table.cohort_month AS cohort_month,
DATE_DIFF (DATE_TRUNC(Date(stories.time_ts), month), cohort_month, Month) as period
FROM `bigquery-public-data.hacker_news.stories` AS stories
LEFT JOIN cohort_table AS cohort_table 
ON stories.author = cohort_table.author
WHERE EXTRACT( YEAR FROM DATE(stories.time_ts)) = 2014 AND stories.author IS NOT NULL
ORDER BY 1,2
)

SELECT
*
FROM active_table

In [10]:
active_table

,author,active_month,cohort_month,period
0,0-o,2014-08-01,2014-08-01,0
1,0-o,2014-09-01,2014-08-01,1
2,0010111,2014-03-01,2014-03-01,0
3,001sky,2014-01-01,2014-01-01,0
4,001sky,2014-01-01,2014-01-01,0
...,...,...,...,...
291269,zzzsh,2014-08-01,2014-08-01,0
291270,zzzviad,2014-01-01,2014-01-01,0
291271,zzzxxx,2014-08-01,2014-08-01,0
291272,zzzzz_,2014-03-01,2014-03-01,0


In [11]:
#Define cohort table

%%bigquery --project revoufsda-350705 cohort_size_table

#Define cohort table
WITH cohort_table AS(
SELECT
author,
MIN(DATE_TRUNC(Date(time_ts),Month)) as cohort_month,
FROM `bigquery-public-data.hacker_news.stories`
WHERE EXTRACT( YEAR FROM DATE(time_ts)) = 2014 AND author IS NOT NULL
GROUP BY 1
ORDER BY 2
),

#Define active month and Return Period User
active_table AS(
SELECT
stories.author AS author,
DATE_TRUNC(Date(stories.time_ts), Month) as active_month,
cohort_table.cohort_month AS cohort_month,
DATE_DIFF (DATE_TRUNC(Date(stories.time_ts), month), cohort_month, Month) as period
FROM `bigquery-public-data.hacker_news.stories` AS stories
LEFT JOIN cohort_table AS cohort_table 
ON stories.author = cohort_table.author
WHERE EXTRACT( YEAR FROM DATE(stories.time_ts)) = 2014 AND stories.author IS NOT NULL
ORDER BY 1,2
),

#Calculate cohort size
cohort_size_table AS(
SELECT
cohort_month,
COUNT(DISTINCT author) as cohort_size
FROM active_table
GROUP BY 1
ORDER BY 1
)

SELECT
*
FROM cohort_size_table

In [12]:
cohort_size_table

,cohort_month,cohort_size
0,2014-01-01,10668
1,2014-02-01,6610
2,2014-03-01,6073
3,2014-04-01,4952
4,2014-05-01,4064
5,2014-06-01,3828
6,2014-07-01,3993
7,2014-08-01,3585
8,2014-09-01,3477
9,2014-10-01,3718


In [13]:
#Get retention user for each period

%%bigquery --project revoufsda-350705 retention_table

#Define cohort table
WITH cohort_table AS
(
SELECT
author,
MIN(DATE_TRUNC(Date(time_ts),Month)) as cohort_month,
FROM `bigquery-public-data.hacker_news.stories`
WHERE EXTRACT( YEAR FROM DATE(time_ts)) = 2014 AND author IS NOT NULL
GROUP BY 1
ORDER BY 2
),

#Define active month and rturn period of the user
active_table AS
(
SELECT
stories.author AS author,
DATE_TRUNC(Date(stories.time_ts), Month) as active_month,
cohort_table.cohort_month AS cohort_month,
DATE_DIFF (DATE_TRUNC(Date(stories.time_ts), month), cohort_month, Month) as period
FROM `bigquery-public-data.hacker_news.stories` AS stories
LEFT JOIN cohort_table AS cohort_table 
ON stories.author = cohort_table.author
WHERE EXTRACT( YEAR FROM DATE(stories.time_ts)) = 2014 AND stories.author IS NOT NULL
ORDER BY 1,2
),

#Calculate cohort size
cohort_size_table AS
(
SELECT
cohort_month,
COUNT(DISTINCT author) as cohort_size
FROM active_table
GROUP BY 1
ORDER BY 1
),

#Define retention table
retention_table AS 
(
SELECT
cohort_month,
A.period,
COUNT(DISTINCT author) as active_author,
C.cohort_size,
FROM active_table AS A
JOIN Cohort_Size_Table AS C USING (cohort_month)
GROUP BY 1,2,4
ORDER BY 1,2
)

SELECT
*
FROM retention_table


In [14]:
retention_table

,cohort_month,period,active_author,cohort_size
0,2014-01-01,0,10668,10668
1,2014-01-01,1,4083,10668
2,2014-01-01,2,3843,10668
3,2014-01-01,3,3471,10668
4,2014-01-01,4,3161,10668
...,...,...,...,...
73,2014-10-01,1,511,3718
74,2014-10-01,2,381,3718
75,2014-11-01,0,3308,3308
76,2014-11-01,1,451,3308


In [15]:
#Define final retention table user for each period

%%bigquery --project revoufsda-350705 final_retention_table

#Define cohort table
WITH cohort_table AS
(
SELECT
author,
MIN(DATE_TRUNC(Date(time_ts),Month)) as cohort_month,
FROM `bigquery-public-data.hacker_news.stories`
WHERE EXTRACT( YEAR FROM DATE(time_ts)) = 2014 AND author IS NOT NULL
GROUP BY 1
ORDER BY 2
),

#Define active month and rturn period of the user
active_table AS
(
SELECT
stories.author AS author,
DATE_TRUNC(Date(stories.time_ts), Month) as active_month,
cohort_table.cohort_month AS cohort_month,
DATE_DIFF (DATE_TRUNC(Date(stories.time_ts), month), cohort_month, Month) as period
FROM `bigquery-public-data.hacker_news.stories` AS stories
LEFT JOIN cohort_table AS cohort_table 
ON stories.author = cohort_table.author
WHERE EXTRACT( YEAR FROM DATE(stories.time_ts)) = 2014 AND stories.author IS NOT NULL
ORDER BY 1,2
),

#Calculate cohort size
cohort_size_table AS
(
SELECT
cohort_month,
COUNT(DISTINCT author) as cohort_size
FROM active_table
GROUP BY 1
ORDER BY 1
),

#Define retention table
retention_table AS 
(
SELECT
cohort_month,
A.period,
COUNT(DISTINCT author) as active_author,
C.cohort_size,
FROM active_table AS A
JOIN Cohort_Size_Table AS C USING (cohort_month)
GROUP BY 1,2,4
ORDER BY 1,2
)

#Final retention table
SELECT
cohort_month AS first_post_month,
period AS active_post_month,
active_author AS number_of_users,
active_author/cohort_size*100 AS retention_rate
FROM retention_table
ORDER BY 1,2

In [16]:
final_retention_table

,first_post_month,active_post_month,number_of_users,retention_rate
0,2014-01-01,0,10668,100.000000
1,2014-01-01,1,4083,38.273341
2,2014-01-01,2,3843,36.023622
3,2014-01-01,3,3471,32.536558
4,2014-01-01,4,3161,29.630671
...,...,...,...,...
73,2014-10-01,1,511,13.743948
74,2014-10-01,2,381,10.247445
75,2014-11-01,0,3308,100.000000
76,2014-11-01,1,451,13.633615
